<a href="https://colab.research.google.com/github/thearcanist/Digits-Recognizer/blob/master/digitrecognizer1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_df = pd.read_csv('digits/train.csv') 
test_df = pd.read_csv('digits/test.csv')

In [0]:
y_train = train_df['label']
x_train = train_df.drop(['label'], axis = 1)

In [0]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state = 2)

In [0]:
x_train = x_train / 255.0
test_df = test_df / 255.0
x_val = x_val / 255.0

In [0]:
x_train = x_train.values.reshape(-1, 28, 28, 1)
test_df = test_df.values.reshape(-1, 28, 28, 1)
x_val = x_val.values.reshape(-1, 28, 28, 1)

In [0]:
y_train = to_categorical(y_train, num_classes = 10)

In [0]:
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (5, 5), padding = 'same', activation = 'relu', input_shape = (28, 28, 1)))
model.add(Conv2D(filters = 32, kernel_size = (5, 5), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = 'sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(10, activation = 'softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 28, 28, 64)        1664      
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 28, 28, 32)        51232     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 7, 7, 64)          0         
__________

In [0]:
optimizer = RMSprop(lr = 0.001, rho = 0.9, epsilon = 1e-08, decay = 0.0)

In [0]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
model.fit(x_train, y_train, epochs = 3, shuffle = True, validation_split = 0.33)

Train on 22511 samples, validate on 11089 samples
Epoch 1/3
22511/22511 [==============================] - 321s 14ms/step - loss: 0.0796 - acc: 0.9766 - val_loss: 0.0494 - val_acc: 0.9848
Epoch 2/3
22511/22511 [==============================] - 321s 14ms/step - loss: 0.0604 - acc: 0.9818 - val_loss: 0.0500 - val_acc: 0.9830
Epoch 3/3
22511/22511 [==============================] - 321s 14ms/step - loss: 0.0504 - acc: 0.9849 - val_loss: 0.0365 - val_acc: 0.9887


In [0]:
predictions = model.predict(test_df)

In [0]:
model.evaluate(test_df, predictions)

28000/28000 [==============================] - 89s 3ms/step


[0.06416146097012929, 0.983]

In [0]:
evaluation

[0.029553460624734205, 1.0]

In [0]:
predictions = np.argmax(predictions,axis = 1)

AxisError: ignored

In [0]:
predictions = pd.Series(predictions,name="Label")

In [0]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),predictions],axis = 1)

In [0]:
submission.to_csv("digits/submissions.csv",index=False)